# Week 4 - Static Attitude Determination

This week, we dive into the fascinating world of static attitude determination. Here, the focus is on taking a snapshot of multiple directional observations—like the sun’s heading, magnetic field direction, and star positioning—to compute a spacecraft's 3D orientation. Essentially, we’re using known vectors in both the spacecraft and reference frames to figure out “which way is up” in space.

We’ll cover both classic and modern algorithms that are fundamental in the field of attitude determination. These include:

- **TRIAD Method**: A foundational, straightforward technique for calculating attitude based on two vector observations.
- **Devenport’s q-Method**: A method that minimizes a cost function to estimate attitude, using quaternions as the rotation representation.
- **QUEST (QUaternion ESTimator)**: An optimized version of the q-method, designed for real-time applications and quick computations.
- **OLAE (Optimal Linear Attitude Estimation)**: A linearized approach that simplifies calculations, particularly useful in scenarios with multiple observations.

Throughout the module, we’ll not only dive into how these methods work, but also discuss the pros and cons of each, especially in terms of computational efficiency and accuracy.

<ins>**Learning Objectives**</ins>

- **Determine attitude from a series of heading measurements**: Learn how to combine various directional observations to calculate a reliable orientation in 3D space.
- **Describe various classical and modern algorithms used in attitude determination**: Gain an understanding of the main methods used in both theoretical and practical applications.
- **Derive the fundamental attitude coordinate properties of rigid bodies**: Develop a mathematical grasp of the key properties that govern the rotational dynamics of rigid bodies in space.

---

In [ ]:
# Import Relevant Libraries
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

# 4.1) Attitude Determination Problem Statement

Attitude determination involves finding the orientation of an object (such as a spacecraft) in space, using instantaneous measurements of specific reference directions. This topic focuses on determining attitude without relying on dynamic filtering or rate measurements (e.g., gyroscopes), which are covered in more advanced topics like Kalman filters and require estimation theory. Here, the aim is to determine the orientation based solely on direct observations.


**<ins>Problem Outline</ins>**

- **Objective**: Determine the orientation (attitude) of a body using known reference directions. At a given instant, we take several measurements (e.g., sun, magnetic field, or star directions) to compute the 3D orientation.
- **Challenge**: Measurements provide partial orientation information; to resolve full 3D attitude, multiple independent observations are needed. This leads to scenarios of being "under-sensed" (insufficient data for a complete attitude) or "over-sensed" (more data than necessary).
  
**<ins>Key Assumptions</ins>**

1. **Fixed Position**: We assume knowledge of the object’s position within its environment. Without this, any directional observation (e.g., “the sun is to my right”) would lack context.
2. **Known Environment**: The orientation computations require known reference frames, such as the Earth-Centered Inertial (ECI) frame for measurements like the sun or magnetic field directions.

**<ins>Measurement Requirements</ins>**

- **Minimum Requirements for 3D Attitude**: Attitude in 3D space has three degrees of freedom, requiring at least two independent direction vectors (each providing two pieces of information).
- **Heading Information**: Each directional observation is a unit vector, meaning it only provides two independent pieces of data (e.g., azimuth and elevation), not three. Thus, one vector is insufficient, and two vectors are typically over-specified.

**<ins>Example Scenarios</ins>**

- **2D Orientation**: In 2D (like being rotated in a plane), knowing one direction (e.g., facing North) is enough if the object’s position is fixed.
- **3D Orientation with Floating Object**: In a space station, knowing that a docking port is to your right doesn’t fully define orientation since you could still rotate around the axis pointing to the port.

**<ins>Measurement Process</ins>**

1. **Observations in Body Frame**: Measure known directions (e.g., sun or magnetic field) relative to the object’s body frame.
2. **Environmental Reference in Inertial Frame**: Convert these observations to the inertial frame for comparison. For example, if the sun’s direction is known in ECI coordinates, compare the observed direction with the known direction.

**<ins>Error Sources</ins>**

1. **Measurement Noise**: Sensor inaccuracies, quantization errors, and digital noise impact readings.
2. **Environmental Uncertainties**: 
   - **Magnetic Field**: The Earth’s magnetic field varies and is not precisely predictable.
   - **Sun Position**: Small uncertainties exist due to Earth’s orbit and satellite positioning errors.
   
   These uncertainties complicate the precise determination of the body’s orientation.

**<ins>Mathematical Requirements</ins>**

1. **Mapping Observations to Attitude**: To determine attitude, we compute the Direction Cosine Matrix (DCM) or other attitude representations (e.g., quaternions, CRPs) that best align the measured directions in the body frame with their expected orientations in the inertial frame.
2. **Optimization Needs**: Due to noise and over-specification, we rarely achieve a perfect match. Methods like least squares optimization are used to minimize error and achieve the best-fit attitude solution.

**<ins>Conclusion</ins>**

The problem of attitude determination requires accurately interpreting direction measurements in the body frame relative to known inertial directions. The objective is to compute an orientation that best aligns these vectors, balancing error when more measurements are available than necessary for a complete 3D orientation. Various methods and algorithms exist to address this, each suited to different configurations of measurements and specific mission requirements.


# 4.2) TRIAD Method

The Vector Triad Method is a deterministic approach to estimate a spacecraft's attitude at a given instant using two vector observations. This method introduces an intermediate frame, or "triad" frame, to simplify the problem. By aligning a third frame with both the inertial frame (e.g., stars or magnetic field) and the body frame (sensor measurements), the method allows a straightforward matrix solution to attitude estimation. Here’s an overview of the method and its key steps.

**<ins>Why Use the Vector Triad Method?</ins>**

- **Simple and Efficient**: The triad method is computationally inexpensive, making it suitable for missions with limited processing power.
- **Reliable for Limited Measurements**: When only two measurements are available (e.g., sun and magnetic field directions), this method provides a quick way to estimate attitude.
- **Optimal Choice of Measurement**: For the triad method, the measurement with the highest accuracy (usually the sun heading) is assigned as the primary reference axis. Less accurate measurements, such as the magnetic field, are used as secondary references.
le.


**<ins>Steps in the Vector Triad Method</ins>**

1. **Define Two Frames**:
   - **Inertial Frame**: Denoted as the $n$ frame, representing known directions in space, such as the position of the sun or Earth's magnetic field.
   - **Body Frame**: Denoted as the $b$ frame, representing measurements in the body’s coordinate system, such as sensor readings of the sun and magnetic field.
   - **Triad Frame**: An intermediate frame denoted as $t$, which helps to connect the inertial and body frames.

<br>

2. **Align the Primary Axis**:
   - Select the most accurate measurement (e.g., the sun heading) and align it with the first axis ($\mathbf{t}_1$) of the triad frame.
   - If the sun vector in the body frame is $\mathbf{s}_b$ and the sun vector in the inertial frame is $\mathbf{s}_n$, then:
     $$ \mathbf{t}_1 = \mathbf{s} $$

<br>

3. **Calculate the Orthogonal Vector for the Second Axis**:
   - Use the secondary measurement (e.g., magnetic field) to define a second axis orthogonal to the first axis.
   - For example, if $\mathbf{m}_b$ is the magnetic field in the body frame and $\mathbf{m}_n$ is the magnetic field in the inertial frame, calculate $\mathbf{t}_2$ as:
     $$ \mathbf{t}_2 = \frac{\mathbf{s} \times \mathbf{m}}{|\mathbf{s} \times \mathbf{m}|} $$
   - Normalize $\mathbf{t}_2$ to make it a unit vector.

<br>

4. **Define the Third Axis**:
   - Complete the triad by defining the third axis as the cross product of the first two:
     $$ \mathbf{t}_3 = \mathbf{t}_1 \times \mathbf{t}_2 $$

<br>

5. **Construct Transformation Matrices**:
   - We can compute the **T frame direction axes** using both **B** and **N** frame components:

     **Body Frame Triad Vectors**
     $$
     \mathbf{^B}\hat{t}_1 = \mathbf{^B}\hat{s}
     $$
     $$
     \mathbf{^B}\hat{t}_2 = \frac{\mathbf{^B}\hat{s} \times \mathbf{^B}\hat{m}}{|\mathbf{^B}\hat{s} \times \mathbf{^B}\hat{m}|}
     $$
     $$
     \mathbf{^B}\hat{t}_3 = \mathbf{^B}\hat{t}_1 \times \mathbf{^B}\hat{t}_2
     $$

     **Inertial Frame Triad Vectors**
     $$
     \mathcal{^N}\hat{t}_1 = \mathcal{^N}\hat{s}
     $$
     $$
     \mathcal{^N}\hat{t}_2 = \frac{\mathcal{^N}\hat{s} \times \mathcal{^N}\hat{m}}{|\mathcal{^N}\hat{s} \times \mathcal{^N}\hat{m}|}
     $$
     $$
     \mathcal{^N}\hat{t}_3 = \mathcal{^N}\hat{t}_1 \times \mathcal{^N}\hat{t}_2
     $$

   - In the absence of measurement errors, both sets of **Triad frame representations** should be the same.

   - Construct the **Body to Triad Matrix** ($[\bar{\mathbf{B}}\mathbf{T}]$) and **Inertial to Triad Matrix** ($[\mathcal{N}\mathbf{T}]$):
     $$
     [\bar{\mathbf{B}}\mathbf{T}] = \begin{bmatrix} \mathbf{^B}\hat{t}_1 & \mathbf{^B}\hat{t}_2 & \mathbf{^B}\hat{t}_3 \end{bmatrix}
     $$
     $$
     [\mathcal{N}\mathbf{T}] = \begin{bmatrix} \mathcal{^N}\hat{t}_1 & \mathcal{^N}\hat{t}_2 & \mathcal{^N}\hat{t}_3 \end{bmatrix}
     $$

<br>

6. **Calculate the Attitude Matrix**:
   - The final step is to compute the attitude matrix, $[\bar{\mathbf{B}}\mathbf{T}]$, that transforms from the inertial frame to the body frame:
     $$
     \bar{\mathbf{B}}\mathbf{N} = [\bar{\mathbf{B}}\mathbf{T}] \cdot [\mathcal{N}\mathbf{T}]^T
     $$
     - This product provides the full attitude matrix, which can then be used to extract desired attitude parameters (Euler angles, MRPs, etc.).

<br>


**<ins>Important Considerations</ins>**

- **Measurement Accuracy**: The triad method assumes that the first measurement vector (e.g., sun heading) is more accurate. Aligning this vector with the primary axis ($\mathbf{t}_1$) uses its full information, while the secondary measurement provides additional orientation constraints.
- **Non-Collinearity of Measurements**: The two measurement vectors must not be collinear, as this would fail to constrain the third axis. For example, if the sun vector and magnetic field vector are aligned, the system would lose orientation around that axis.

**<ins>Practical Usage and Limitations</ins>**

- **Efficiency**: This method is fast and computationally light, making it suitable for real-time systems or constrained spacecraft systems.
- **Usage on Spacecraft**: The triad method has been implemented on various spacecraft where simplicity is prioritized, particularly for missions without advanced filtering methods.
- **Accuracy and Redundancy**: While effective, the method does not use all available data when only two measurements are used. As missions typically involve more sensors, more advanced algorithms like QUEST or Kalman filters can be employed to improve attitude estimation accuracy and handle dynamic environments.

**<ins>Summary</ins>**

The vector triad method is a foundational approach in attitude determination, especially valuable for its simplicity and direct use of two sensor measurements. It is a fundamental tool in aerospace engineering for quick and reliable orientation estimation when minimal data is available.

# 4.3) Wahba's Problem Defintion

# 4.4) Devenport's q-Method

# 4.5) QUEST

# 4.6) OLAE

# 4.7) Kinematics Final Assignment